In [ ]:
from pandas import read_csv, get_dummies, Series,DataFrame
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
# Data Preparation
data1=read_csv('/content/exams (3).csv')#reading
# data1.head()
# data1.info()
mean=data1['math score'].mean() # finding value to replace
data1['math score']=data1['math score'].fillna(mean) #replace null with mean value

data1['gender'] = data1['gender'].map({'male':1, 'female':0}) #encoding
data1['lunch'] = data1['lunch'].map({'standard':1, 'free/reduced':0}) #encoding
data1['test preparation course'] = data1['test preparation course'].map({'completed':1, 'none':0}) #encoding
# data1.info()
data2 = get_dummies(data1, columns = ['race/ethnicity','parental level of education']) #encoding
# data2.info()
X = data2.drop('test preparation course', axis = 1) # Features
Y = data2['test preparation course'] # Labels
X_scaled = StandardScaler().fit_transform(X) #scaling
# X_scaled
DataFrame(X_scaled)
X_train, X_test, Y_train, Y_test = train_test_split( X_scaled, Y, test_size = 0.25, random_state = 100)
X_train,Y_train = SMOTE(random_state = 101).fit_resample(X_train,Y_train)

In [ ]:
data1['gender'].unique()
data1['race/ethnicity'].unique()
data1['parental level of education'].unique()
data1['lunch'].unique()
data1['test preparation course'].unique()

array(['completed', 'none'], dtype=object)

In [ ]:
# Decision Tree Classifier (method#1)
from sklearn import tree
#from sklearn.tree import DecisionTreeClassifier as dt
DT_classifier1 = tree.DecisionTreeClassifier(criterion = 'entropy', max_depth = 15)# classifier building
m=DT_classifier1.fit(X_train, Y_train) # training
Y_pred1 = m.predict(X_test) # testing
# Important_features = Series(DT_classifier1.feature_importances_, index=list(X)).sort_values(ascending=False) # what are the important features?
# print(Important_features)

In [ ]:
# Random Forest Classifier (method 1)
#from sklearn.ensemble import RandomForestClassifier as rf
from sklearn import ensemble
RF_classifier1 = ensemble.RandomForestClassifier(n_estimators=30, criterion='entropy', max_features='auto', random_state=1)  # building model
RF_classifier1.fit(X_train,Y_train)#training
Y_pred2 = RF_classifier1.predict(X_test) # testing
Important_features = Series(RF_classifier1.feature_importances_, index=list(X)).sort_values(ascending=False)
print(Important_features)

In [ ]:
# Support Vector Machine (SVM) Classifier (method 1)
from sklearn import svm
#import sklearn as sk
SVM_classifier1 = svm.SVC(kernel='linear', random_state=0, )  # building classifier
SVM_classifier1.fit(X_train, Y_train) # training
Y_pred3= SVM_classifier1.predict(X_test)  #testing

In [ ]:
# Acuracy and confusion matrix
from sklearn import metrics
Accuracy=metrics.accuracy_score(Y_test, Y_pred1) # calculating accuaracy
print("Accuracy: ", Accuracy) # Is this a good metric??
con_matrix = metrics.confusion_matrix(Y_test, Y_pred1)
print (con_matrix)
recall = metrics.recall_score(Y_test, Y_pred1)
print (recall)
percision=metrics.precision_score(Y_test, Y_pred1)
print(percision)

Accuracy:  0.636
[[111  59]
 [ 32  48]]
0.6
0.4485981308411215


In [ ]:
#using Grid Search for decision tree Classifier(method#3)
from imblearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
#DT_classifier2 = tree.DecisionTreeClassifier(criterion = 'entropy') # building classfier
DT_classifier2 = Pipeline([('balancing', SMOTE(random_state = 101)),('classification', tree.DecisionTreeClassifier(criterion = 'entropy') ) ]) # bulilding classifier
depth = {'classification__max_depth': [2,3,4,5,10,15,20,25,30,35]}
grid_search1 = GridSearchCV(estimator=DT_classifier2, param_grid=depth, scoring='recall', cv=4)
grid_search1.fit(X_scaled, Y) # training & testing

best_depth = grid_search1.best_params_
print(best_depth)
best_result = grid_search1.best_score_
print(best_result)

{'classification__max_depth': 2}
0.7317484222604705


In [ ]:
#DT_classifier2.get_params().keys()

In [ ]:
# using Grid Search for Random Forest Classifier (method 3)
#RF_classifier2 = ensemble.RandomForestClassifier(criterion='entropy', max_features='auto', random_state=1) # building model
RF_classifier2 = Pipeline([('balancing', SMOTE(random_state = 101)),('classification', ensemble.RandomForestClassifier(criterion='entropy', max_features='auto', random_state=1))]) # building classifier
no_trees = {'classification__n_estimators': [200, 250, 300, 350, 400, 450]}
grid_search2 = GridSearchCV(estimator=RF_classifier2, param_grid=no_trees, scoring='recall', cv=5)
grid_search2.fit(X_scaled, Y)#training and testing

best_no_trees = grid_search2.best_params_
print(best_no_trees)
best_result = grid_search2.best_score_
print(best_result)

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomFor

{'classification__n_estimators': 200}
0.5253731343283582


In [ ]:
#RF_classifier2.get_params().keys()

In [ ]:
#  Support Vector Classifier (method#3)
from imblearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
SVM_classifier2 = Pipeline([('balancing', SMOTE(random_state = 101)),('classification', svm.SVC(random_state=10) ) ]) # bulilding classifier
kernels_c = {'classification__kernel': ['linear','poly','rbf','sigmoid'], 'classification__C': [.001,.01,.1,1,10,100]}
grid_search3 = GridSearchCV(estimator=SVM_classifier2, param_grid=kernels_c, scoring='recall', cv=5)
grid_search3.fit(X_scaled, Y)#training and testing

best_parameters = grid_search3.best_params_
print(best_parameters)
best_result = grid_search3.best_score_
print(best_result)

{'classification__C': 0.001, 'classification__kernel': 'poly'}
0.9402985074626866
